In [2]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2021-07-15 05:03:29--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M  4.47MB/s    in 12s     

2021-07-15 05:03:42 (4.75 MB/s) - ‘/tmp/Chessman-image-dataset.zip’ saved [60684125/60684125]



In [3]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode = 'nearest',
    validation_split=0.1) # set validation split

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  
])
model.layers[0].trainable = False

In [6]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [7]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              epochs=50,batch_size = 64,
                              verbose=2)

Epoch 1/50
63/63 - 95s - loss: 9.2670 - accuracy: 0.4549 - val_loss: 3.9821 - val_accuracy: 0.4231
Epoch 2/50
63/63 - 86s - loss: 1.3699 - accuracy: 0.7154 - val_loss: 1.2172 - val_accuracy: 0.6538
Epoch 3/50
63/63 - 86s - loss: 1.0470 - accuracy: 0.7455 - val_loss: 1.2692 - val_accuracy: 0.7115
Epoch 4/50
63/63 - 86s - loss: 0.5843 - accuracy: 0.8357 - val_loss: 0.9864 - val_accuracy: 0.6923
Epoch 5/50
63/63 - 86s - loss: 0.5093 - accuracy: 0.8457 - val_loss: 1.3343 - val_accuracy: 0.7308
Epoch 6/50
63/63 - 86s - loss: 0.6056 - accuracy: 0.8377 - val_loss: 1.2542 - val_accuracy: 0.5577
Epoch 7/50
63/63 - 86s - loss: 0.7983 - accuracy: 0.8156 - val_loss: 0.7898 - val_accuracy: 0.7308
Epoch 8/50
63/63 - 86s - loss: 0.9778 - accuracy: 0.8116 - val_loss: 0.9075 - val_accuracy: 0.7692
Epoch 9/50
63/63 - 86s - loss: 0.6784 - accuracy: 0.8457 - val_loss: 1.0448 - val_accuracy: 0.7885
Epoch 10/50
63/63 - 86s - loss: 0.5704 - accuracy: 0.8657 - val_loss: 0.8674 - val_accuracy: 0.7692
Epoch 11/